In [1]:
import pandas as pd
import multiprocessing
pd.set_option('display.max_colwidth', None)
import re
import os
os.chdir(r'D:\Хакатон')
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_excel("ИКРБС_2020-2024_гг..xlsx")

In [3]:
df.fillna('Отсутствует', inplace=True)

In [4]:
df.drop(["№ п/п", "Регистрационный номер"],axis=1,inplace=True)

In [5]:
df['text_data'] = df['Наименование отчёта'] + ' ' + df['Реферативное описание отчета'] + ' ' + df['Ключевые слова']



In [22]:
df

,Наименование отчёта,Реферативное описание отчета,Полученные (ожидаемые) результаты и их возможная практическая значимость (применимость),Ключевые слова,ГРНТИ,Первая рубрика ГРНТИ,text_data
0,Исследования и разработка новых и совершенствование существующих технологических процессов обработки давлением с целью улучшения качества машиностроительных заготовок,"Проведены теоретические и экспериментальные исследования процессов выдавливания осесимметричных деталей, а также исследования процессов вытяжки и отбортовки окрашенных и оцинкованных стальных листовых заготовок. Разработаны и исследованы технологические процессы штамповки осесимметричных деталей повышенной точности. Разработаны технологические процессы холодного выдавливания с применением методов планирования и статистической обработки данных многофакторных экспериментов. Установлены связи между механическими характеристиками изделий, а также комплекс механических характеристик при выдавливании деталей из отожженных стальных заготовок с содержанием углерода 0,6% со степенью деформации за переход 46%.",Отсутствует,ХОЛОДНОЕ ВЫДАВЛИВАНИЕ; РОТАЦИОННОЕ ВЫДАВЛИВАНИЕ; СТАЛЬНЫЕ ЛИСТОВЫЕ ЗАГОТОВКИ; ОСЕСИММЕТРИЧНЫЕ ДЕТАЛИ; ВЫТЯЖКА;,55.16.21: Ротационная вытяжка,55.16.21: Ротационная вытяжка,"Исследования и разработка новых и совершенствование существующих технологических процессов обработки давлением с целью улучшения качества машиностроительных заготовок Проведены теоретические и экспериментальные исследования процессов выдавливания осесимметричных деталей, а также исследования процессов вытяжки и отбортовки окрашенных и оцинкованных стальных листовых заготовок. Разработаны и исследованы технологические процессы штамповки осесимметричных деталей повышенной точности. Разработаны технологические процессы холодного выдавливания с применением методов планирования и статистической обработки данных многофакторных экспериментов. Установлены связи между механическими характеристиками изделий, а также комплекс механических характеристик при выдавливании деталей из отожженных стальных заготовок с содержанием углерода 0,6% со степенью деформации за переход 46%. ХОЛОДНОЕ ВЫДАВЛИВАНИЕ; РОТАЦИОННОЕ ВЫДАВЛИВАНИЕ; СТАЛЬНЫЕ ЛИСТОВЫЕ ЗАГОТОВКИ; ОСЕСИММЕТРИЧНЫЕ ДЕТАЛИ; ВЫТЯЖКА;"
1,Исследование технической возможности работы судовых дизелей на сырой нефти Якутского месторождения,"Определены некоторые свойства сырой нефти Якутского месторождения. Проведено сравнительное испытание показателей работы топливной аппаратуры на нефти и дизельном топливе. Проведены пуски дизеля 2Ч8,5/11 на дизельном топливе и нефти. Проведено сравнение процессов наброса нагрузки на дизеле 2Ч8,5/11, работающем на дизельном топливе и нефти, сняты нагрузочные характеристики дизеля. Сняты винтовые характеристики дизеля 6LI60PNS на дизельном топливе и нефти.",Отсутствует,ТОПЛИВНАЯ АППАРАТУРА; СУДОВЫЕ ДИЗЕЛИ; СЫРЫЕ НЕФТИ; ДИЗЕЛЬНЫЕ ТОПЛИВА;,55.42.29: Дизели; 55.45.29: Судовые энергетические установки,55.42.29: Дизели,"Исследование технической возможности работы судовых дизелей на сырой нефти Якутского месторождения Определены некоторые свойства сырой нефти Якутского месторождения. Проведено сравнительное испытание показателей работы топливной аппаратуры на нефти и дизельном топливе. Проведены пуски дизеля 2Ч8,5/11 на дизельном топливе и нефти. Проведено сравнение процессов наброса нагрузки на дизеле 2Ч8,5/11, работающем на дизельном топливе и нефти, сняты нагрузочные характеристики дизеля. Сняты винтовые характеристики дизеля 6LI60PNS на дизельном топливе и нефти. ТОПЛИВНАЯ АППАРАТУРА; СУДОВЫЕ ДИЗЕЛИ; СЫРЫЕ НЕФТИ; ДИЗЕЛЬНЫЕ ТОПЛИВА;"
2,"Морфология слизистых оболочек пищеварительного тракта и кожи при различных инфекционных заболеваниях бактериального и вирусного происхождения (дифтерия, кокковые инфекции ротоносоглотки, аденовирусные инфекции, цитомегаловирусная инфекция, корь, ветряная оспа, ВИЧ-инфекция)","Установлены особенности реакций различных популяций клеток слизистых оболочек пищеварительной системы и различных слоев кожи, 

In [6]:
train_df, test_df = train_test_split(df, test_size=250, train_size=250)

In [7]:
sentence_example = df.iloc[1]["text_data"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]


In [8]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Исследование технической возможности работы судовых дизелей на сырой нефти Якутского месторождения Определены некоторые свойства сырой нефти Якутского месторождения. Проведено сравнительное испытание показателей работы топливной аппаратуры на нефти и дизельном топливе. Проведены пуски дизеля 2Ч8,5/11 на дизельном топливе и нефти. Проведено сравнение процессов наброса нагрузки на дизеле 2Ч8,5/11, работающем на дизельном топливе и нефти, сняты нагрузочные характеристики дизеля. Сняты винтовые характеристики дизеля 6LI60PNS на дизельном топливе и нефти. ТОПЛИВНАЯ АППАРАТУРА; СУДОВЫЕ ДИЗЕЛИ; СЫРЫЕ НЕФТИ; ДИЗЕЛЬНЫЕ ТОПЛИВА;
-----------------
Токены: ['Исследование', 'технической', 'возможности', 'работы', 'судовых', 'дизелей', 'на', 'сырой', 'нефти', 'Якутского', 'месторождения', 'Определены', 'некоторые', 'свойства', 'сырой', 'нефти', 'Якутского', 'месторождения', '.', 'Проведено', 'сравнительное', 'испытание', 'показателей', 'работы', 'топливной', 'аппаратуры', 'на', 'нефт

In [9]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [10]:
tokenize_sentence(sentence_example)

['исследован',
 'техническ',
 'возможн',
 'работ',
 'судов',
 'дизел',
 'сыр',
 'нефт',
 'якутск',
 'месторожден',
 'определ',
 'некотор',
 'свойств',
 'сыр',
 'нефт',
 'якутск',
 'месторожден',
 'провед',
 'сравнительн',
 'испытан',
 'показател',
 'работ',
 'топливн',
 'аппаратур',
 'нефт',
 'дизельн',
 'топлив',
 'провед',
 'пуск',
 'дизел',
 '2ч8,5/11',
 'дизельн',
 'топлив',
 'нефт',
 'провед',
 'сравнен',
 'процесс',
 'наброс',
 'нагрузк',
 'дизел',
 '2ч8,5/11',
 'работа',
 'дизельн',
 'топлив',
 'нефт',
 'снят',
 'нагрузочн',
 'характеристик',
 'дизел',
 'снят',
 'винтов',
 'характеристик',
 'дизел',
 '6LI60PNS',
 'дизельн',
 'топлив',
 'нефт',
 'топливн',
 'аппаратур',
 'судов',
 'дизел',
 'сыр',
 'нефт',
 'дизельн',
 'топлив']

In [11]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))

In [12]:
features = vectorizer.fit_transform(train_df["text_data"])

D:\Jupiter\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [33]:
X = features

In [43]:
# Создать целевую переменную
y = df['ГРНТИ'][:250]



In [44]:


# Разделить данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [45]:

# Обучить модель классификации
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [46]:

y_pred = model.predict(X_test)



In [49]:
from sklearn.metrics import accuracy_score

In [50]:

accuracy = accuracy_score(y_test, y_pred)
print('Точность:', accuracy)

Точность: 0.0


In [58]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df["ГРНТИ"])


LogisticRegression(random_state=0)

In [59]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model", LogisticRegression(random_state=0))
]
)

In [60]:
model_pipeline.fit(train_df["ГРНТИ"], train_df["text_data"])

D:\Jupiter\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x0000022BDFA1D940>)),
                ('model', LogisticRegression(random_state=0))])

In [61]:

accuracy = accuracy_score(y_true=test_df["text_data"], y_pred=model_pipeline.predict(test_df["ГРНТИ"]))
print('Точность:', accuracy)

Точность: 0.0


In [57]:
prec, rec, thresholds = precision_recall_curve(y_true=test_df["text_data"], probas_pred=model_pipeline.predict_proba(test_df["ГРНТИ"]))


ValueError: multiclass format is not supported

In [ ]:
plot_precision_recall_curve(estimator=model_pipeline, X=test_df["ГРНТИ"], y=test_df["text_data"])

In [ ]:
np.where(prec > 0.95)

In [ ]:
thresholds[374]


In [ ]:
precision_score(y_true=test_df["text_data"], y_pred=model_pipeline.predict_proba(test_df["ГРНТИ"])[:, 1] > thresholds[374])

In [ ]:
recall_score(y_true=test_df["text_data"], y_pred=model_pipeline.predict_proba(test_df["ГРНТИ"])[:, 1] > thresholds[374])

In [ ]:
grid_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model", 
     GridSearchCV(
        LogisticRegression(random_state=0),
        param_grid={'C': [0.1, 1, 10.]},
        cv=3,
         verbose=4
        )
    )
])


In [ ]:
grid_pipeline.fit(train_df["ГРНТИ"], train_df["text_data"])

In [ ]:
model_pipeline_c_10 = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model", LogisticRegression(random_state=0, C=10.))
]
)


In [ ]:
model_pipeline_c_10.fit(train_df["ГРНТИ"], train_df["text_data"])

In [ ]:
prec_c_10, rec_c_10, thresholds_c_10 = precision_recall_curve(y_true=test_df["text_data"], probas_pred=model_pipeline_c_10.predict_proba(test_df["ГРНТИ"])[:, 1])

In [ ]:
np.where(prec_c_10 > 0.95)

In [ ]:
precision_score(y_true=test_df["text_data"], y_pred=model_pipeline_c_10.predict_proba(test_df["ГРНТИ"])[:, 1] > thresholds_c_10[316])


In [ ]:
recall_score(y_true=test_df["text_data"], y_pred=model_pipeline_c_10.predict_proba(test_df["ГРНТИ"])[:, 1] > thresholds_c_10[316])

In [21]:
recall_score

<function sklearn.metrics._classification.recall_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>